\# 學習各種Class（類別）的進階用法

In [1]:
# 作用域（Scope）及命名空間（Namespace）
def scope_test():
    def do_local():
        spam = "local spam"

    def do_nonlocal():
        nonlocal spam
        spam = "nonlocal spam"

    def do_global():
        global spam
        spam = "global spam"

    spam = "test spam"
    do_local()
    print("After Local Assignment:", spam)
    do_nonlocal()
    print("After Nonlocal Assignment:", spam)
    do_global()
    print("After Global Assignment:", spam)

scope_test()
print("In Global Scope:", spam)

After Local Assignment: test spam
After Nonlocal Assignment: nonlocal spam
After Global Assignment: nonlocal spam
In Global Scope: global spam


務必要了解，作用域是按文本被決定的：在模組中定義的函式，其全域作用域便是該模組的命名空間，無論函式是從何處或以什麼別名被呼叫。另一方面，對名稱的實際搜尋是在執行時期動態完成的——但是，語言定義的發展，正朝向在「編譯」時期的靜態名稱解析（static name resolution），所以不要太依賴動態名稱解析（dynamic name resolution）！（事實上，局部變數已經是靜態地被決定。）

一個 Python 的特殊癖好是——假如沒有 global 或 nonlocal 陳述式的效果——名稱的賦值（assignment）都會指向最內層作用域。賦值不會複製資料——它們只會把名稱連結至物件。刪除也是一樣：陳述式 del x 會從區域作用域參照的命名空間移除 x 的連結。事實上，引入新名稱的所有運算都使用區域作用域：特別是 import 陳述式和函式定義，會連結區域作用域內的模組或函式名稱。

global 陳述式可以用來表示特定變數存活在全域作用域，應該被重新綁定到那裡；nonlocal 陳述式表示特定變數存活在外圍作用域內，應該被重新綁定到那裡。

In [2]:
# 結構（Struct）
class Employee:
    pass

John = Employee()
John.name = 'John Doe'
John.dept = "Computer Lab"
John.salary = 1000

如果有一種資料型別，類似於 Pascal 的「record」或 C 的「struct」，可以將一些有名稱的資料項目捆綁在一起，有時候這會很有用。其實一個空白的 class definition 就可以勝任

In [3]:
# 疊代器（Iterator）
s = 'abc'
it = iter(s)
it

In [4]:
next(it)

'a'

In [5]:
next(it)

'b'

In [6]:
next(it)

'c'

In [7]:
next(it)

StopIteration: 

疊代器的使用在 Python 中處處可見且用法一致。在幕後，for 陳述式會在容器物件上呼叫 iter()。該函式回傳一個疊代器物件，此物件定義了 \__next__() method，而此 method 會逐一存取容器中的元素。當元素用盡時，\__next__() 將引發 StopIteration 例外，來通知 for 終止迴圈。你可以使用內建函式 next() 來呼叫 \__next__() method

In [8]:
class Reverse:
    def __init__(self, data):
        self.data = data
        self.index = len(data)

    def __iter__(self):
        return self
    
    def __next__(self):
        if self.index == 0:
            raise StopIteration
        self.index = self.index - 1
        return self.data[self.index]

In [9]:
rev = Reverse('spam')
iter(rev)

In [10]:
for char in rev:
    print(char)

m
a
p
s


看過疊代器協定的幕後機制後，在你的 class 加入疊代器的行為就很容易了。定義一個 \__iter__() method 來回傳一個帶有 \__next__() method 的物件。如果 class 已定義了 \__next__()，則 \__iter__() 可以只回傳 self

In [11]:
# 產生器（Generator）
def reverse(data):
    for index in range(len(data) - 1, -1, -1):
        yield data[index]


In [12]:
for char in reverse('golf'):
    print(char)

f
l
o
g


產生器是一個用於建立疊代器的簡單而強大的工具。它們的寫法和常規的函式一樣，但當它們要回傳資料時，會使用 yield 陳述式。每次在產生器上呼叫 next() 時，它會從上次離開的位置恢復執行（它會記得所有資料值以及上一個被執行的陳述式）。

任何可以用產生器來完成的事，也能用以 class 為基礎的疊代器來完成，如同前一節的描述。而讓產生器的程式碼更為精簡的原因是，\__iter__() 和 \__next__() method 會自動被建立。

另一個關鍵的特性在於，區域變數和執行狀態會在每次呼叫之間自動被儲存。這使得該函式比使用 self.index 和 self.data 這種實例變數的方式更容易編寫且更為清晰。

除了會自動建立 method 和儲存程式狀態，當產生器終止時，它們還會自動引發 StopIteration。這些特性結合在一起，使建立疊代器能與編寫常規函式一樣容易。